<a href="https://colab.research.google.com/github/CamelGoong/DataScienceLab/blob/main/%5B%EB%B0%95%ED%95%B4%EA%B7%A0%5DRNN_%EC%84%B8%EC%85%98%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Input 및 기본 Parameter 설정

In [94]:
import numpy as np

k = 4 # dimensions of input vector
d = 3 # number of hidden nodes
epochs = 100
lr = 0.001 # learning rate

# Input data
x1 = np.array([1, 0, 0, 0]).reshape(k,1)
x2 = np.array([0, 1, 0, 0]).reshape(k,1)
x3 = np.array([0, 0, 1, 1]).reshape(k,1)

# Y_true data
y_true1 = np.array([0, 1, 0, 0]).reshape(k,1)
y_true2 = np.array([0, 0, 1, 0]).reshape(k,1)
y_true3 = np.array([0, 0, 0, 1]).reshape(k,1)

RNN Class

In [95]:
class RNN:
  def __init__(self, wx, wh, bh, wy, by): # wx: d*k wh: d*d bh: d*1 wy: k*d by: k*1
    self.params = [wx, wh, bh, wy, by]
    self.grads = [np.zeros_like(wx), np.zeros_like(wh), np.zeros_like(bh), np.zeros_like(wy), np.zeros_like(by)]
    self.saved = None

  def forward(self, x, h_prev): # x: k*1 
    wx, wh, bh, wy, by = self.params
    a = np.matmul(wh,h_prev) + np.matmul(wx, x) + bh # a: d*1
    h_next = np.tanh(a) # h_next: d*1
    zt = np.matmul(wy, a) + by # yt: k*1
    exp_zt = np.exp(zt)
    yt = exp_zt / np.sum(exp_zt) # softmax

    self.saved = (x, h_prev, h_next)
    self.result = yt
    return h_next

  def backward(self, y_true):
    wx, wh, bh, wy, by = self.params
    x, h_prev, h_next = self.saved
    yt = self.result

    dby = y_true - yt
    dwy = np.matmul(dby, h_next.T)
    dht = np.matmul(wy.T, by)
    dbh = dht * (1-h_next**2)
    dwh = np.matmul(dbh, h_prev.T)
    dwx = np.matmul(dbh, x.T)
    self.grads = [dwx, dwh, dbh, dwy, dby]
    return self.grads

Gradients update

In [96]:
# parameter initialize
wx = np.random.rand(d, k)
wh = np.random.rand(d, d)
bh = np.random.rand(d, 1)
wy = np.random.rand(k, d)
by = np.random.rand(k, 1)
parameters = [wx, wh, bh, wy, by]

for epochs in range(epochs):
# Gradient initialize
  dwx = np.zeros((d, k))
  dwh = np.zeros((d, d))
  dbh = np.zeros((d, 1))
  dwy = np.zeros((k, d))
  dby = np.zeros((k, 1))
  grads = [dwx, dwh, dbh, dwy, dby]
  h_prev = np.zeros((d,1))
  
  RNN_t1 =  RNN(parameters[0], parameters[1], parameters[2], parameters[3], parameters[4])
  h_prev = RNN_t1.forward(x1, h_prev)
  for i in range(5):
    grads[i] += RNN_t1.backward(y_true1)[i]

  RNN_t2 = RNN(parameters[0], parameters[1], parameters[2], parameters[3], parameters[4])
  h_prev = RNN_t2.forward(x2, h_prev)
  for i in range(5):
    grads[i] += RNN_t1.backward(y_true1)[i]

  RNN_t3 = RNN(parameters[0], parameters[1], parameters[2], parameters[3], parameters[4])
  h_prev = RNN_t3.forward(x3, h_prev)
  for i in range(5):
    grads[i] += RNN_t1.backward(y_true1)[i]
  
# Gradient update
  for i in range(5):
    parameters[i] = parameters [i] - lr * grads[i]

  if (epochs + 1) % 10 == 0:
    print("Epochs", epochs + 1)
    print("Wx: \n", parameters[0], end ='\n')
    print("Wh: \n", parameters[1], end ='\n')
    print("Bh: \n", parameters[2], end ='\n')
    print("Wy: \n", parameters[3], end ='\n')
    print("By: \n", parameters[4], end ='\n')
    print("-----"*20)  


Epochs 10
Wx: 
 [[0.22325668 0.74631873 0.86096017 0.69779623]
 [0.12320522 0.25422179 0.35557777 0.71859301]
 [0.09042026 0.84724665 0.46433366 0.1074329 ]]
Wh: 
 [[0.79608697 0.50726406 0.91924087]
 [0.33104346 0.24487272 0.17007127]
 [0.6833745  0.41086545 0.68070266]]
Bh: 
 [[0.25058122]
 [0.55275863]
 [0.31257073]]
Wy: 
 [[0.13067262 0.15752692 0.91397781]
 [0.52250455 0.47491857 0.86135481]
 [0.95763166 0.29306969 0.49569719]
 [0.70815942 0.36411157 0.23169046]]
By: 
 [[ 0.0160661 ]
 [-0.00103542]
 [ 0.51049447]
 [ 0.91080969]]
----------------------------------------------------------------------------------------------------
Epochs 20
Wx: 
 [[0.19510732 0.74631873 0.86096017 0.69779623]
 [0.1135948  0.25422179 0.35557777 0.71859301]
 [0.0781303  0.84724665 0.46433366 0.1074329 ]]
Wh: 
 [[0.79608697 0.50726406 0.91924087]
 [0.33104346 0.24487272 0.17007127]
 [0.6833745  0.41086545 0.68070266]]
Bh: 
 [[0.22243186]
 [0.54314822]
 [0.30028078]]
Wy: 
 [[0.13237686 0.15988949 0.91548